# Day 28 - REINFORCE

## Intuition Behind Policy Gradients

* Directly learning a policy allows an agent to handle continuous action spaces naturally
* Policy gradient methods are able to learn stochastic policies explicitly
* Policy estimation is the simplest, most direct form of RL, as it directly learns the final goal: The policy
* REINFORCE uses Monte Carlo returns, which can avoid bootstrapping stability issues, but can be quite slow and introduce a lot of variance

## Mathematical Foundation: Policy Gradient Theorem

## Implementation: REINFORCE in Atari Breakout (Gymnasium + PyTorch)

## Training and Debugging the REINFORCE Agent

## Next Steps: Improving and Extending REINFORCE